In [1]:
import geopandas as gpd
import pandas as pd

In [2]:
lineas_ffcc = pd.read_csv('../carto/insumos/ffcc/ffcc.csv',usecols = ['id_linea','nombre']) 
lineas_ffcc

,id_linea,nombre
0,16,FFCC ROCA
1,44,FFCC_BELG_SUR
2,49,FFCC_SUAREZ
3,132,FFCC SAR MERLO - LOBOS
4,133,FFCC SAR MORENO - MERCEDES
5,135,FFCC_TREN_COSTA
6,144,FFCC SAR
7,149,FFCC URQUIZA
8,160,FCC MITRE_TIGRE
9,172,FFCC_MITRE


In [18]:
ign = gpd.read_file('../carto/insumos/ffcc/lineas ig no en mintrans.geojson')
ign = ign.reindex(columns = ['nam','geometry'])
ign.columns = ['nombre_parada','geometry']
ign['nombre_linea'] = 'Roca'
ign['nombre_ramal'] = 'Roca - Universitario'

est_ramal_chas = ['Domselaar','Chascomús','Brandsen','Altamirano','Jeppener']
ign.loc[ign.nombre_parada.isin(est_ramal_chas),'nombre_ramal'] = 'Roca - Chascomus'
ign.head(2)

,nombre_parada,geometry,nombre_linea,nombre_ramal
0,Altamirano,POINT (-58.15130 -35.36120),Roca,Roca - Chascomus
1,Jeppener,POINT (-58.19810 -35.27760),Roca,Roca - Chascomus


In [20]:
mintrans = gpd.read_file('../carto/insumos/ffcc/rmba-ferrocarril-estaciones/')
mintrans = mintrans.reindex(columns = ['Línea','ETIQUETA','geometry'])
mintrans.columns = ['nombre_linea','nombre_parada','geometry']
mintrans.head(2)

,nombre_linea,nombre_parada,geometry
0,Mitre,San Martin,POINT (-58.53165 -34.57368)
1,Mitre,San Andres,POINT (-58.54055 -34.56368)


In [49]:
ffcc = pd.concat([mintrans,ign])

In [50]:
lineas = gpd.read_file('../carto/insumos/ffcc/rmba-ferrocarril-lineas')
lineas = lineas.reindex(columns = ['Linea','Descrip','geometry'])
lineas.columns = ['linea','ramal','geometry']
lineas['ramal'] = lineas['linea'] + ' - ' + lineas['ramal']
lineas.geometry = lineas.geometry.to_crs('EPSG:3857').buffer(500).to_crs('EPSG:4326')


In [51]:
ffcc = gpd.sjoin(ffcc,lineas,how='left')
ffcc.loc[ffcc.nombre_ramal.isnull(),'nombre_ramal']=ffcc.loc[ffcc.nombre_ramal.isnull(),'ramal']
ffcc = ffcc.reindex(columns = ['nombre_linea', 'nombre_parada', 'geometry', 'nombre_ramal'])

In [52]:
ffcc.head()

,nombre_linea,nombre_parada,geometry,nombre_ramal
0,Mitre,San Martin,POINT (-58.53165 -34.57368),Mitre - Retiro - José León Suárez
1,Mitre,San Andres,POINT (-58.54055 -34.56368),Mitre - Retiro - José León Suárez
2,Mitre,Villa Ballester,POINT (-58.55608 -34.55091),Mitre - Retiro - José León Suárez
3,Mitre,Malaver,POINT (-58.54785 -34.55744),Mitre - Retiro - José León Suárez
4,Mitre,Chilavert,POINT (-58.56694 -34.54217),Mitre - Retiro - José León Suárez


In [54]:
ffcc.to_file('../carto/insumos/ffcc/paradas_ffcc.geojson',driver='GeoJSON')

In [55]:
lineas

,linea,ramal,geometry
0,Belgrano Sur,Belgrano Sur - Línea G: Buenos Aires - Tapiales,"POLYGON ((-58.63868 -34.77074, -58.63980 -34.7..."
1,Mitre,Mitre - Retiro - José León Suárez,"POLYGON ((-58.37992 -34.59254, -58.37994 -34.5..."
2,Mitre,Mitre - Tren de la Costa,"POLYGON ((-58.53516 -34.44902, -58.53517 -34.4..."
3,Mitre,Mitre - Victoria - Capilla del Señor,"POLYGON ((-58.53870 -34.46027, -58.53983 -34.4..."
4,Belgrano Sur,Belgrano Sur - Línea M: Puente Alsina - Aldo B...,"POLYGON ((-58.68917 -34.69557, -58.68926 -34.6..."
5,Mitre,Mitre - Retiro - Tigre,"POLYGON ((-58.57469 -34.43440, -58.57475 -34.4..."
6,Roca,Roca - Temperley - Bosques - Villa Elisa,"POLYGON ((-58.39141 -34.78031, -58.39132 -34.7..."
7,Roca,Roca - Plaza Constitución -La Plata,"POLYGON ((-58.37604 -34.63118, -58.37599 -34.6..."
8,Roca,Roca - Temperley - Alejandro Korn,"POLYGON ((-58.39215 -34.77638, -58.39208 -34.7..."
9,Roca,Roca - Constitución - Temperley,"POLYGON ((-58.39986 -34.78799, -58.39986 -34.7..."


In [ ]:
trx = gpd.read_file('../carto/insumos/ffcc/borrar.shp')
trx = trx.reindex(columns = ['id_linea','geometry'])

In [ ]:
trx.head()

In [ ]:
buffers = ffcc.geometry.to_crs('EPSG:3857').buffer(500).to_crs('EPSG:4326')
buffers = gpd.GeoDataFrame(ffcc[['nombre_linea','nombre_parada']],geometry = buffers)

In [ ]:
buffers

In [ ]:
join = gpd.sjoin(trx,buffers)

In [ ]:
tabla = pd.DataFrame(join.reindex(columns = ['index_right','id_linea']).groupby('index_right').apply(lambda s: s.id_linea.value_counts()))
tabla = tabla.reset_index()
tabla.columns = ['index_right','id_linea','n']
tabla.set_index('index_right',inplace=True)
tabla

In [ ]:
ffcc_con_id_linea = ffcc.merge(tabla,left_index=True,right_index=True,how='left')

In [ ]:
ffcc_con_id_linea.shape

In [ ]:
ffcc_con_id_linea = ffcc_con_id_linea.merge(lineas_ffcc,on='id_linea',how='left')

In [ ]:
ffcc_con_id_linea[ffcc_con_id_linea['nombre'].isnull()]